In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import SVD, evaluate
from surprise import NMF
from surprise import BaselineOnly
from surprise import KNNBasic

In [38]:
def mask(column,threshold):
    counts = pd.value_counts(column)
    mask = column.isin(counts[counts > threshold].index)
    column[~mask] = "others"
    return column

In [39]:
bookings_data = pd.read_csv('BookingsData.csv')

In [40]:
bookings_data = bookings_data.drop_duplicates()
bookings_data["Customer_Name"] = bookings_data["Customer_Name"].str.lower()
bookings_data["Product_Family"] = bookings_data["Product_Family"].str.lower()
bookings_data["BOARD_GEO"] = bookings_data["BOARD_GEO"].str.lower()
bookings_data["Customer_Name"] = bookings_data["Customer_Name"].replace('\s+', ' ', regex=True)
bookings_data["TX_Date_mod"] = pd.to_datetime(bookings_data['TX_Date'])
bookings_data = bookings_data[(bookings_data["TX_Date_mod"] > '2012')]
bookings_data.loc[(bookings_data['Product_Family'] == "lan desk (other)"),"Product_Family"] = 'others'

# if trying classification
bookings_data['Product_Family'] = mask(bookings_data['Product_Family'], 50)
bookings_data['PRODUCTCODE'] = mask(bookings_data['PRODUCTCODE'], 20)
bookings_data.count()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Customer_Name      141799
Account_ID          14265
Bookings Amount    141799
TX_Date            141799
Seats              141799
GEO                 98553
BOARD_GEO          141799
VERTICAL           125251
Product_Family     141799
Deal Type          141799
Product Type       141799
Product Segment    135956
Contract Start     113938
Contract End        89477
PRODUCTCODE        141799
FAMILY             139802
NAME               141799
DESCRIPTION         87502
TX_Date_mod        141799
dtype: int64

In [41]:
bookings_data['Seats_mod'] = np.where(bookings_data['Seats'] > 0, 1, (np.where(bookings_data['Seats'] == 0, 0, -1)))

In [42]:
company_standards = pd.read_csv('standardization2.csv')
company_standards_2 = pd.read_csv('standardization2_2.csv')
print(company_standards.shape)

bookings_data = bookings_data.merge(company_standards, left_on='Customer_Name', right_on='Similar_Name', how='inner')
bookings_data = bookings_data.rename(index=str, columns={"Customer_Name_x": "Customer_Name_old1", 
                                                         "Customer_Name_y": "Customer_Name"})

bookings_data = bookings_data.merge(company_standards_2, left_on='Customer_Name', right_on='Similar_Name', how='inner')
bookings_data = bookings_data.rename(index=str, columns={"Customer_Name_x": "Customer_Name_old2", 
                                                         "Customer_Name_y": "Customer_Name"})

print(len(bookings_data.Customer_Name.unique()))
print(len(bookings_data.Customer_Name_old2.unique()))

(2572, 3)
113
149


In [43]:
table = bookings_data.pivot_table(values=['Seats_mod'], index=['Customer_Name'], columns=['Product_Family'],
                     aggfunc='sum').fillna(0)
table

Seats_mod                                                 \
Product_Family    anti virus application manager application virtualization   
Customer_Name                                                                 
a-katsastus              0.0                 0.0                        0.0   
acm                      0.0                 0.0                        0.0   
american express         0.0                 0.0                        0.0   
amundi                   0.0                 0.0                        0.0   
axians                   0.0                 0.0                        0.0   
baxter ag                0.0                 0.0                        0.0   
bcd travel               0.0                 0.0                        0.0   
bitdefender srl          0.0                 0.0                        0.0   
bnp paribas              0.0                 0.0                        0.0   
canada life              0.0                 0.0                        0.0   
capgemini                0.0                 0.0                        0.0   
capita                   0.0                 0.0                        0.0   
carlson wagonlit         0.0                 0.0                        0.0   
cellent                  0.0                 0.0                        0.0   
centro                   0.0                 0.0                        0.0   
cms                     12.0                 0.0                        0.0   
cobham                   0.0                 0.0                        0.0   
commerzbank ag           0.0                 0.0                        0.0   
computer sciences        0.0                 0.0                        2.0   
coop                     0.0                 0.0                        0.0   
cosco                    0.0                 0.0                        0.0   
crane co.                0.0                 0.0                        0.0   
crown equipment          0.0                 0.0                        0.0   
csc                      0.0                 0.0                        0.0   
csl behring              0.0                 0.0                        0.0   
dell                     0.0                 0.0                        0.0   
dexia                    0.0                 0.0                        0.0   
direct energy            0.0                 0.0                        0.0   
disa                     0.0                 0.0                        0.0   
dts                      0.0                 0.0                        0.0   
...                      ...                 ...                        ...   
raymond james            0.0                 0.0                        0.0   
raytheon                 0.0                 0.0                        0.0   
rexel                    0.0                 0.0                        0.0   
ring                     2.0                 0.0                        0.0   
saic                     0.0                 0.0                        0.0   
sercel                   0.0                 0.0                        0.0   
serco                    0.0                 0.0                        0.0   
siemens ag               0.0                 0.0                        0.0   
sita                     6.0                 0.0                        0.0   
societe generale         0.0                 0.0                        0.0   
sodexo                   0.0                 0.0                        0.0   
softronic                0.0                 0.0                        0.0   
sonepar                  0.0                 0.0                        0.0   
stalwart                 0.0                 0.0                        0.0   
switch                   0.0                 0.0                        0.0   
t-systems                0.0                12.0                        0.0   
taby kommun              0.0                 0.0                     

In [44]:
bookings_data["TX_Date_mod"] = pd.to_datetime(bookings_data['TX_Date'])

In [45]:
bookings_data.loc[(bookings_data['Product_Family'] == "lan desk (other)"),"Product_Family"] = 'others'

In [46]:
from scipy.sparse.linalg import *
U, sigma, Vt = svds(table)

sigma = np.diag(sigma)

all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = table.columns, index= table.index)

preds_df

Seats_mod                      \
Product_Family       anti virus application manager   
Customer_Name                                         
a-katsastus       -2.436551e-04       -1.466241e-02   
acm               -2.086257e-04       -4.460397e-03   
american express   8.514908e-04        1.417642e-01   
amundi             6.054399e-02       -2.915530e-02   
axians             7.988711e-03        1.565555e+00   
baxter ag          2.070374e-02        4.687530e-02   
bcd travel         4.190957e-02       -1.888199e-02   
bitdefender srl    2.261331e-06       -2.852984e-06   
bnp paribas        9.258485e-03        6.020613e-01   
canada life        1.943427e-01        3.851957e+00   
capgemini          1.319426e-02        2.049053e-01   
capita             1.044518e-02       -2.080672e-02   
carlson wagonlit   1.687511e-02        3.354113e+00   
cellent            3.261062e-05        6.495362e-03   
centro             2.067746e-02        8.002092e-02   
cms                2.430081e-02        2.151586e-02   
cobham             2.936763e-03       -4.519958e-04   
commerzbank ag     5.085873e-04        1.464302e-01   
computer sciences -8.432141e-04       -2.428046e-01   
coop               2.626556e-02        5.849221e-01   
cosco              6.843459e-03        2.958364e-02   
crane co.          3.696010e-05        3.174770e-01   
crown equipment    1.748759e-02        1.398697e+00   
csc               -1.064953e-01        1.329727e+00   
csl behring        3.325788e-05        3.570420e-03   
dell               3.393402e-05        2.729604e-04   
dexia              2.652960e-03        3.601230e-01   
direct energy     -4.873103e-04       -2.932482e-02   
disa              -4.873103e-04       -2.932482e-02   
dts                7.593048e-04        1.369002e-01   
...                         ...                 ...   
raymond james      2.909833e-03        1.961081e-01   
raytheon          -1.261052e-04        1.757973e-02   
rexel              3.165916e-05        3.037751e-04   
ring               1.364579e-03        7.437232e-03   
saic              -1.467049e-04       -1.559229e-02   
sercel            -5.908488e-05        4.574196e-03   
serco              4.847513e-05       -4.649426e-04   
siemens ag        -2.121608e-03       -1.148407e-01   
sita               4.162472e-02        1.844781e+00   
societe generale   9.741572e-03        1.087353e+00   
sodexo             4.539162e-04        6.570904e-02   
softronic          6.643561e-03        1.304385e+00   
sonepar            5.893951e-02       -2.895310e-02   
stalwart          -1.096448e-03       -6.598084e-02   
switch             4.709453e-04        4.222928e-02   
t-systems         -7.158254e-03       -1.001651e-01   
taby kommun        3.986136e-03        7.826310e-01   
tbwa               2.341361e-03       -1.648901e-02   
teleperformance    1.500677e-01        5.019273e-02   
tullett prebon    -1.598741e-03       -1.007288e-01   
tyro payments     -4.385792e-03       -2.639234e-01   
unicef             3.033035e-03        5.967875e-01   
veka ag            2.393380e-02       -1.245107e-02   
villa maria        8.901843e-03       -4.070909e-03   
vodafone           9.417762e-04        9.398808e-01   
vwr                1.716733e-02       -2.567331e-03   
wipro limited      1.342198e-02       -6.169727e-03   
worldpay           1.317390e-02        2.523437e+00   
xerox              4.748421e-04       -7.824095e-02   
yasui kagu        -3.915485e-20       -1.394926e-16   

                                                                          \
Product_Family    application virtualization asset manager       datanow   
Customer_Name                                                              
a-katsastus                     4.838733e-05 -1.970031e-06 -5.538204e-03   
acm                             1.200570e-04 -2.131721e-06 -4.294201e-03   
american express                3.627169e-04  3.561403e-07  7.345572e-03   
amundi                          7.022469e-04 

In [47]:
user_data = table.loc["amundi"].unstack(-1)

prod = np.dot(sigma, Vt)

print(U.shape)
print(sigma.shape)
print(Vt.shape)

print(table.shape)
print(preds_df.shape)
prod.shape

(113, 6)
(6, 6)
(6, 40)
(113, 40)
(113, 40)


(6, 40)

In [48]:
data = bookings_data[["Customer_Name","Product_Family","Seats_mod"]]
data = pd.DataFrame({'count' : data.groupby( ["Customer_Name", "Product_Family" ] ).size()}).reset_index()
data["count"].describe()

count     399.000000
mean       22.644110
std        97.220925
min         1.000000
25%         2.000000
50%         4.000000
75%        15.000000
max      1440.000000
Name: count, dtype: float64

In [49]:
data1 = Dataset.load_from_df(data, Reader(rating_scale=(1,5000)))

In [22]:
#data1.split(n_folds=5)
#trainset, testset = train_test_split(data1, test_size=.25)
trainset = data1.build_full_trainset()

# svd
algo = SVD()
algo.fit(trainset)
evaluate(algo, data1, measures=['RMSE'])
#predictions_svd = algo.test(testset)
# print("-----break----")
# print(accuracy.rmse(predictions_svd))

# nmf
algo = NMF()
algo.fit(trainset)
evaluate(algo, data1, measures=['RMSE'])
#predictions_nmf = algo.test(testset)
# print("-----break----")
# print(accuracy.rmse(predictions_nmf))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 4980.7380
------------
Fold 2
RMSE: 4908.2729
------------
Fold 3
RMSE: 4933.2311
------------
Fold 4
RMSE: 4985.5726
------------
Fold 5
RMSE: 4989.2406
------------
------------
Mean RMSE: 4959.4110
------------
------------
Evaluating RMSE of algorithm NMF.

------------
Fold 1
RMSE: 56.4959
------------
Fold 2
RMSE: 123.8676
------------
Fold 3
RMSE: 166.9318
------------
Fold 4
RMSE: 23.4266
------------
Fold 5
RMSE: 21.8171
------------
------------
Mean RMSE: 78.5078
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [56.495877143472754,
                             123.86764496600887,
                             166.93178198949647,
                             23.42664103433405,
                             21.817098749789547]})

In [23]:
algo.predict("amundi","desktopnow")

Prediction(uid='amundi', iid='desktopnow', r_ui=None, est=5.406726246206629, details={'was_impossible': False})

In [24]:
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)

algo.fit(trainset)
evaluate(algo, data1, measures=['RMSE'])

Using ALS
Estimating biases using als...
Evaluating RMSE of algorithm BaselineOnly.

------------
Fold 1
Estimating biases using als...
RMSE: 54.9155
------------
Fold 2
Estimating biases using als...
RMSE: 125.4009
------------
Fold 3
Estimating biases using als...
RMSE: 165.1699
------------
Fold 4
Estimating biases using als...
RMSE: 32.4536
------------
Fold 5
Estimating biases using als...
RMSE: 30.9854
------------
------------
Mean RMSE: 81.7851
------------
------------


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


CaseInsensitiveDefaultDict(list,
                           {'rmse': [54.9154619503457,
                             125.40094398759113,
                             165.16994548834552,
                             32.45358861567655,
                             30.985448621999787]})

In [25]:
print('Using SGD')
bsl_options = {'method': 'sgd',
               'learning_rate': .00005,
               }
algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)
evaluate(algo, data1, measures=['RMSE'])

Using SGD
Estimating biases using sgd...
Evaluating RMSE of algorithm BaselineOnly.

------------
Fold 1
Estimating biases using sgd...
RMSE: 54.7921
------------
Fold 2
Estimating biases using sgd...
RMSE: 124.7364
------------
Fold 3
Estimating biases using sgd...
RMSE: 165.6402
------------
Fold 4
Estimating biases using sgd...
RMSE: 24.1050
------------
Fold 5
Estimating biases using sgd...
RMSE: 26.9300
------------
------------
Mean RMSE: 79.2408
------------
------------


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


CaseInsensitiveDefaultDict(list,
                           {'rmse': [54.792104497222844,
                             124.73643242061354,
                             165.64022937404408,
                             24.105004111259408,
                             26.930015320656373]})

In [26]:
bsl_options = {'method': 'als',
               'n_epochs': 20,
               }
sim_options = {'name': 'pearson_baseline'}
algo = KNNBasic(bsl_options=bsl_options, sim_options=sim_options)
algo.fit(trainset)
evaluate(algo, data1, measures=['RMSE'])

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic.

------------
Fold 1
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 74.4616
------------
Fold 2
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 131.0814
------------
Fold 3
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 166.5846
------------
Fold 4
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 81.0758
------------
Fold 5
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 165.8602
------------
------------
Mean RMSE: 123.8127
------------
------------


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


CaseInsensitiveDefaultDict(list,
                           {'rmse': [74.4616315770626,
                             131.08139903816772,
                             166.58458281273184,
                             81.07581862469257,
                             165.86022503124033]})

In [27]:
sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between items
               }
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)
evaluate(algo, data1, measures=['RMSE'])

Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic.

------------
Fold 1
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 51.4653
------------
Fold 2
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 124.1930
------------
Fold 3
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 172.4939
------------
Fold 4
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 51.4566
------------
Fold 5
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 48.3069
------------
------------
Mean RMSE: 89.5831
------------
------------


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


CaseInsensitiveDefaultDict(list,
                           {'rmse': [51.46533307332846,
                             124.19298778136104,
                             172.49387055636132,
                             51.45663064886058,
                             48.30686165334842]})

In [28]:
sim_options = {'name': 'pearson_baseline',
               'shrinkage': 0  # no shrinkage
               }
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)
evaluate(algo, data1, measures=['RMSE'])

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic.

------------
Fold 1
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 76.6489
------------
Fold 2
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 130.7782
------------
Fold 3
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 166.8858
------------
Fold 4
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 81.1376
------------
Fold 5
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 164.6801
------------
------------
Mean RMSE: 124.0261
------------
------------


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


CaseInsensitiveDefaultDict(list,
                           {'rmse': [76.64889625950909,
                             130.7782322955981,
                             166.88579101716164,
                             81.13763358650851,
                             164.6800671352279]})

In [29]:
import turicreate
train_data = turicreate.SFrame(data)
test_data = turicreate.SFrame(data)
popularity_model = turicreate.popularity_recommender.create(train_data, 
                                    user_id='Customer_Name', item_id='Product_Family', target='count')

Recsys training: model = popularity

Preparing data set.

Data has 399 observations with 113 users and 40 items.

Data prepared in: 0.008531s

399 observations to process; with 40 unique items.

In [30]:
popularity_recomm = popularity_model.recommend(users=["amundi","acm","axians"],k=15)
popularity_recomm.print_rows(num_rows=45)

+---------------+-----------------------+--------------------+------+
| Customer_Name |     Product_Family    |       score        | rank |
+---------------+-----------------------+--------------------+------+
|     amundi    | shavlik oem royalties | 60.666666666666664 |  1   |
|     amundi    |        protect        |        57.1        |  2   |
|     amundi    |     security suite    |        50.0        |  3   |
|     amundi    |         ht oem        |        41.5        |  4   |
|     amundi    |          sccm         |        36.0        |  5   |
|     amundi    |       desktopnow      | 28.708333333333332 |  6   |
|     amundi    |       xtraction       | 27.09090909090909  |  7   |
|     amundi    |     heat 2014 saas    | 23.166666666666668 |  8   |
|     amundi    |        datanow        |        20.0        |  9   |
|     amundi    |          emss         | 17.72222222222222  |  10  |
|     amundi    |     managed planet    | 15.333333333333334 |  11  |
|     amundi    |   

In [31]:
#Training the model
item_sim_model = turicreate.item_similarity_recommender.create(train_data, 
                        user_id='Customer_Name', item_id='Product_Family', target='count', similarity_type='cosine')

#Making recommendations
item_sim_model.evaluate(train_data)
item_sim_recomm = item_sim_model.recommend(users=["amundi","acm","axians"],k=5)
item_sim_recomm.print_rows(num_rows=25)

Recsys training: model = item_similarity

Preparing data set.

Data has 399 observations with 113 users and 40 items.

Data prepared in: 0.004326s

Training model from provided data.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.02912s


Precision and recall summary statistics by cutoff
+--------+-------------+----------------+
| cutoff | mean_recall | mean_precision |
+--------+-------------+----------------+
|   1    |     0.0     |      0.0       |
|   2    |     0.0     |      0.0       |
|   3    |     0.0     |      0.0       |
|   4    |     0.0     |      0.0       |
|   5    |     0.0     |      0.0       |
|   6    |     0.0     |      0.0       |
|   7    |     0.0     |      0.0       |
|   8    |     0.0     |      0.0       |
|   9    |     0.0     |      0.0       |
|   10   |     0.0     |      0.0       |
+--------+-------------+----------------+
[10 rows x 3 columns]


Overall RMSE: 99.56748915873104

Per User RMSE (best)
+---------------+--------------------+-------+
| Customer_Name |        rmse        | count |
+---------------+--------------------+-------+
|    parexel    | 0.9535553914419275 |   2   |
+---------------+--------------------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+--

In [32]:
#Training the model
item_sim_model = turicreate.item_similarity_recommender.create(train_data, 
                        user_id='Customer_Name', item_id='Product_Family', target='count', similarity_type="pearson")

#Making recommendations
item_sim_recomm = item_sim_model.recommend(users=["amundi","acm","axians"],k=5)
item_sim_recomm.print_rows(num_rows=25)

Recsys training: model = item_similarity

Preparing data set.

Data has 399 observations with 113 users and 40 items.

Data prepared in: 0.004455s

Training model from provided data.

...

| 17.947ms                            | 49.75            | 19              |

+---------------+-----------------------+--------------------+------+
| Customer_Name |     Product_Family    |       score        | rank |
+---------------+-----------------------+--------------------+------+
|     amundi    | shavlik oem royalties | 60.666666666666664 |  1   |
|     amundi    |        protect        | 57.01071621576942  |  2   |
|     amundi    |     security suite    | 49.92975507179896  |  3   |
|     amundi    |         ht oem        |        41.5        |  4   |
|     amundi    |          sccm         |        36.0        |  5   |
|      acm      | shavlik oem royalties | 60.666666666666664 |  1   |
|      acm      |     security suite    | 49.976753771305084 |  2   |
|      acm      |     patch manager     | 43.78454810651867  |  3   |
|      acm      |         ht oem        |        41.5        |  4   |
|      acm      |          sccm         |        36.0        |  5   |
|     axians    | shavlik oem royalties | 60.61738050977389  |  1   |
|     axians    |   

| 26.329ms                            | 100              | 40              |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

In [33]:
#Training the model
item_sim_model = turicreate.item_similarity_recommender.create(train_data, 
                        user_id='Customer_Name', item_id='Product_Family', target='count', similarity_type='jaccard')

#Making recommendations
item_sim_recomm = item_sim_model.recommend(users=["amundi","acm","axians"],k=5)
item_sim_recomm.print_rows(num_rows=25)

Generating candidate set for working with new users.

Finished training in 0.028276s

Recsys training: model = item_similarity

Preparing data set.

Data has 399 observations with 113 users and 40 items.

...

| 21.752ms                            | 100              | 40              |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.023004s